### 涨跌停数据统计分析

#### 导入相关模块

In [1]:
import tushare as ts
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import datetime as dt
import time 
from pyecharts.charts import Bar,Line,Scatter,Pie
from pyecharts import options as opts
from pyecharts.render import make_snapshot
from snapshot_phantomjs import snapshot
from pyecharts.globals import ThemeType

#### 设置输出显示
##### 单元格设置
+ 允许一个单元格有多个输出
+ 显示所有列
+ 不换行

In [27]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # 允许一个单元格有多个输出
pd.set_option('display.max_columns', None)  # 显示所有列，None无限制
pd.set_option("display.max_rows", None)  # 显示所有行，None无限制
pd.set_option('expand_frame_repr', False)  # 不换行

##### 设置中英文列名对照
+ 按tushare中对应函数的输出参数顺序

In [35]:
llCname = {
    'pct_chg': '涨跌幅',
    'amp': '振幅',
    'fc_ratio': '封单额/日成交额',
    'fl_ratio': '封单手数/流通股本',
    'fd_amount': '封单额',
    'first_time': '首次涨停',
    'last_time': '最后封板时',
    'open_times': '打开次数',
    'strth': '涨跌停强度',
    'limit': 'D跌停U涨停',
}
outfields = 'trade_date,ts_code,close,pct_chg,fc_ratio,fl_ratio,first_time,fd_amount,last_time,open_times,strth,limit'

#### 设置数据目录

In [5]:
Stock_Data_Dir=Path.home()/'Stock_Data/Tushare'
if (not Stock_Data_Dir.exists()):
    Stock_Data_Dir.mkdir()
out_csv_dir=Stock_Data_Dir/'csv'
if(not out_csv_dir.exists()):  #Path.exists()
    out_csv_dir.mkdir()

#### 设置token并初始化接口¶

In [6]:
tokenfile=Stock_Data_Dir/'Token'/'token.pk'
with open(tokenfile,'rb') as f :
    mytoken=pickle.load(f)
ts.set_token(mytoken)   
pro = ts.pro_api()

#### 设置查询时间

In [55]:
sdate,query_date='20190513','20200328'

### 获取数据
#### 当日所有涨跌停股票

In [37]:
limit_list = pro.limit_list(trade_date=query_date)
limit_list.rename(columns=llCname)

,trade_date,ts_code,name,close,涨跌幅,振幅,封单额/日成交额,封单手数/流通股本,封单额,首次涨停,最后封板时,打开次数,涨跌停强度,D跌停U涨停
0,20200327,600228.SH,ST昌九,8.63,4.99,0.00,11479.4120,2.2643,4.715605e+07,09:25:02,09:25:02,0,100.000000,U
1,20200327,688228.SH,N开普云,90.06,51.97,25.31,0.0040,0.0029,3.728484e+04,09:25:04,09:25:04,0,100.000000,U
2,20200327,600462.SH,*ST九有,1.56,4.70,0.00,920.6027,1.5388,1.281353e+07,09:25:02,09:25:02,0,97.221090,U
3,20200327,002501.SZ,*ST利源,1.59,5.30,1.32,67.2117,0.3208,5.789190e+06,09:25:00,09:25:00,0,75.417206,U
4,20200327,600983.SH,惠而浦,5.20,9.94,0.00,292.5118,1.4016,3.883220e+07,09:25:04,09:25:04,0,75.237915,U
5,20200327,002269.SZ,美邦服饰,2.22,9.90,0.00,173.5252,0.7068,3.942576e+07,09:25:00,09:25:00,0,71.073380,U
6,20200327,002571.SZ,德力股份,5.36,10.06,0.00,142.2209,3.9474,6.012682e+07,09:25:00,09:25:00,0,69.977730,U
7,20200327,002094.SZ,青岛金王,4.55,9.90,0.00,84.1706,1.7704,5.352147e+07,09:25:00,09:25:00,0,67.945970,U
8,20200327,300307.SZ,慈星股份,4.96,9.98,0.67,28.7442,0.8681,3.415597e+07,13:57:03,13:57:03,0,67.247826,U
9,20200327,603608.SH,天创时尚,11.92,9.96,0.00,52.0882,1.3960,6.863071e+07,09:25:02,09:25:02,0,66.823080,U


#### 个股在指定时间段内的涨跌情况

In [38]:
limit603157 = pro.limit_list(start_date=sdate,
                             end_date=query_date,
                             ts_code='603157.SH,000007.SZ',fields=outfields)
limit603157.rename(columns=llCname).sort_values(by='ts_code')

,trade_date,ts_code,close,涨跌幅,封单额/日成交额,封单手数/流通股本,封单额,首次涨停,最后封板时,打开次数,涨跌停强度,D跌停U涨停
1,20200210,000007.SZ,8.33,10.04,14.0717,1.0061,25892706.0,14:25:33,14:44:51,12,49.567474,U
3,20200204,000007.SZ,7.38,-10.00,1342.4928,8.7376,199221952.0,09:25:03,09:25:03,0,100.000000,D
4,20200203,000007.SZ,8.20,-9.99,881.2895,4.3054,109072816.0,09:30:03,09:33:33,6,60.305270,D
8,20191209,000007.SZ,10.56,10.00,11.4832,0.5090,16605601.0,14:50:36,14:52:30,2,50.133770,U
13,20191114,000007.SZ,11.06,10.05,2.1534,0.1978,6760303.5,09:25:03,14:49:24,4,57.665226,U
14,20191113,000007.SZ,10.05,9.96,11.6980,0.3910,12141405.0,10:25:45,10:33:30,8,51.339348,U
0,20200327,603157.SH,4.49,10.05,45.6151,2.2248,14564513.0,09:41:25,09:53:34,1,56.342747,U
2,20200207,603157.SH,5.10,9.91,5.6227,0.7626,5670685.0,14:10:54,14:46:27,21,47.020400,U
5,20200203,603157.SH,4.78,-9.98,74.6085,4.0199,28015582.0,09:25:04,10:04:15,2,64.651850,D
6,20200123,603157.SH,5.31,9.94,1.6694,0.2236,1731060.0,10:00:43,14:42:30,106,48.588116,U


In [21]:
limit07= pro.limit_list(start_date=sdate,end_date=query_date,ts_code='000007.SZ')
limit07.rename(columns=llCname)

,trade_date,ts_code,name,close,涨跌幅,振幅,封单额/日成交额,封单手数/流通股本,封单额,首次涨停,最后封板时,打开次数,涨跌停强度,D跌停U涨停
0,20200210,000007.SZ,全新好,8.33,10.04,11.89,14.0717,1.0061,25892706.0,14:25:33,14:44:51,12,49.567474,U
1,20200204,000007.SZ,全新好,7.38,-10.00,0.00,1342.4928,8.7376,199221952.0,09:25:03,09:25:03,0,100.000000,D


#### 开板情况

In [56]:
stk_limit = pro.stk_limit(ts_code='000007.SZ', start_date=sdate, end_date=query_date)
stk_limit.tail()

,trade_date,ts_code,up_limit,down_limit
211,20190517,000007.SZ,7.32,5.99
212,20190516,000007.SZ,7.46,6.10
213,20190515,000007.SZ,7.43,6.08
214,20190514,000007.SZ,7.28,5.96
215,20190513,000007.SZ,6.84,5.60


In [57]:
daily_fields='ts_code,trade_date,open,high,low,close'
df07=pro.daily(ts_code='000007.SZ', start_date=sdate, end_date=query_date,fields=daily_fields)
df07['up_limit']=stk_limit['up_limit']
df07['down_limit']=stk_limit['down_limit']
df07

,ts_code,trade_date,open,high,low,close,up_limit,down_limit
0,000007.SZ,20200327,8.16,8.30,8.10,8.14,8.95,7.33
1,000007.SZ,20200326,8.30,8.30,8.08,8.14,9.14,7.48
2,000007.SZ,20200325,8.18,8.31,8.15,8.31,8.95,7.33
3,000007.SZ,20200324,8.25,8.29,8.01,8.14,9.05,7.41
4,000007.SZ,20200323,8.20,8.43,8.08,8.23,9.14,7.48
5,000007.SZ,20200320,8.23,8.38,8.17,8.31,9.11,7.45
6,000007.SZ,20200319,8.05,8.30,7.97,8.28,8.90,7.28
7,000007.SZ,20200318,8.12,8.31,8.04,8.09,8.93,7.31
8,000007.SZ,20200317,8.10,8.23,7.52,8.12,8.93,7.31
9,000007.SZ,20200316,8.48,8.59,8.08,8.12,9.24,7.56


In [58]:
cond1=(df07.high==df07.up_limit)&(df07.close!=df07.up_limit)
cond2=(df07.low==df07.down_limit)&(df07.close!=df07.down_limit)
#df07[(cond1&cond2)]
df07[(df07.high==df07.up_limit)|(df07.low==df07.down_limit)]

,ts_code,trade_date,open,high,low,close,up_limit,down_limit
34,000007.SZ,20200210,7.60,8.33,7.43,8.33,8.33,6.81
37,000007.SZ,20200205,6.67,8.11,6.64,7.48,8.12,6.64
38,000007.SZ,20200204,7.38,7.38,7.38,7.38,9.02,7.38
39,000007.SZ,20200203,8.20,8.21,8.20,8.20,10.02,8.20
72,000007.SZ,20191209,9.58,10.56,9.50,10.56,10.56,8.64
82,000007.SZ,20191125,10.50,10.51,9.32,9.96,11.40,9.32
89,000007.SZ,20191114,11.06,11.06,10.30,11.06,11.06,9.05
90,000007.SZ,20191113,9.15,10.05,9.15,10.05,10.05,8.23
111,000007.SZ,20191015,10.45,10.45,9.46,9.64,10.45,8.55
113,000007.SZ,20191011,9.00,9.80,8.86,9.60,9.80,8.02


#### 涨跌停数据基本情况

In [22]:
limit_data.info()
v=[]  
v=np.random.randint(len(limit_data.index),size=10)
limit_data.iloc[v]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22772 entries, 0 to 22771
Data columns (total 14 columns):
trade_date    22772 non-null object
ts_code       22772 non-null object
name          22772 non-null object
close         22772 non-null float64
pct_chg       22772 non-null float64
amp           22772 non-null float64
fc_ratio      22772 non-null float64
fl_ratio      22772 non-null float64
fd_amount     22772 non-null float64
first_time    22772 non-null object
last_time     22772 non-null object
open_times    22772 non-null int64
strth         22772 non-null float64
limit         22772 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 2.4+ MB


,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
9964,20190305,300128.SZ,锦富技术,5.60,10.02,12.38,5.5430,0.6248,3.728605e+07,10:06:54,10:06:54,0,53.205540,U
7938,20190513,002766.SZ,*ST索菱,3.71,-4.87,0.00,10053.4270,8.6068,7.944520e+07,09:25:03,09:25:03,0,100.000000,D
8462,20190225,002876.SZ,三利谱,47.74,10.00,9.77,5.0934,0.2079,5.428038e+06,14:21:45,14:54:09,6,50.749733,U
14728,20200115,300811.SZ,铂科新材,107.76,10.00,0.00,19.4512,5.7877,8.980998e+07,,,0,0.000000,U
4892,20190318,002302.SZ,西部建设,13.61,10.02,10.02,5.9398,0.3036,5.119777e+07,14:05:33,14:05:33,0,55.589160,U
9952,20190606,300127.SZ,银河磁体,18.02,10.01,12.27,37.3094,2.5457,1.041845e+08,09:40:30,09:40:30,0,56.588223,U
1840,20190510,000811.SZ,冰轮环境,8.69,10.00,9.37,0.3420,0.0130,7.377809e+05,14:49:36,14:55:15,3,10.664848,U
11435,20190107,300345.SZ,红宇新材,4.10,9.92,6.70,54.3556,1.1560,1.747188e+07,09:30:24,09:30:24,0,63.793358,U
3719,20190425,002140.SZ,东华科技,12.06,10.04,9.67,1.5512,0.1709,9.060678e+06,09:34:33,14:49:51,59,55.493668,U
3113,20190311,002058.SZ,威 尔 泰,14.01,9.97,9.11,7.6118,0.3045,6.119288e+06,13:49:09,13:51:54,4,51.666405,U


在20190101至20200118期间，共有627只股票没涨跌停


['600811.SH',
 '002457.SZ',
 '601966.SH',
 '002203.SZ',
 '601991.SH',
 '002173.SZ',
 '603308.SH',
 '603728.SH',
 '603955.SH',
 '002344.SZ',
 '600011.SH',
 '600919.SH',
 '002749.SZ',
 '603897.SH',
 '601333.SH',
 '002038.SZ',
 '002061.SZ',
 '688089.SH',
 '601636.SH',
 '600723.SH']

In [13]:
csvfilepath=out_csv_dir/'limit_data.csv'
limit_data=pd.read_csv(str(csvfilepath))
limit_data

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20191107,000004.SZ,国农科技,20.88,10.01,0.00,95.4751,1.9584,33775864.0,09:25:03,09:25:03,0,68.341630,U
1,20190919,000004.SZ,国农科技,22.00,10.00,8.90,21.4926,1.4702,26716448.0,NaN,NaN,0,0.000000,U
2,20190506,000004.SZ,国农科技,22.75,-10.01,8.50,0.0000,0.0000,0.0,09:33:27,14:52:03,13,51.500000,D
3,20190430,000004.SZ,国农科技,25.28,10.01,10.97,4.1339,0.1489,3121650.2,10:05:51,14:52:51,31,49.450832,U
4,20190410,000004.SZ,国农科技,23.84,10.01,0.00,0.2913,0.0012,23840.0,09:25:03,09:25:03,0,65.010190,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22767,20191014,688368.SH,N晶丰,104.36,84.12,23.09,0.1583,0.0002,1606517.9,09:25:00,09:25:00,0,100.000000,U
22768,20191031,688369.SH,N致远,76.90,55.70,24.76,0.1120,0.0002,1174032.4,09:25:03,09:25:03,0,100.000000,U
22769,20190722,688388.SH,嘉元科技,56.66,100.50,140.94,0.3722,0.0006,7846503.5,NaN,NaN,0,0.000000,U
22770,20191105,688389.SH,N普门,19.63,115.71,42.64,0.2979,0.0002,1833540.1,09:25:03,09:25:03,0,100.000000,U


In [ ]:
limit_data[]

In [114]:

pro.limit_list(ts_code='600388.SH',start_date=sdate, end_date=edate)

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit


In [15]:
limit_data[(limit_data.open_times>10)&(limit_data.limit=='U')]

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
3,20190430,000004.SZ,国农科技,25.28,10.01,10.97,4.1339,0.1489,3121650.2,10:05:51,14:52:51,31,49.450832,U
5,20190325,000004.SZ,国农科技,21.67,10.00,5.89,2.4241,0.1332,2396702.0,09:33:09,14:53:18,23,54.359070,U
7,20190903,000005.SZ,世纪星源,3.48,10.13,10.13,3.8327,0.3445,12683556.0,10:37:21,14:47:12,25,50.270493,U
22,20190225,000008.SZ,神州高铁,4.42,9.95,9.95,0.6532,0.0249,2811937.8,13:50:12,14:05:00,16,50.116560,U
26,20190819,000009.SZ,中国宝安,4.74,9.98,5.57,5.6314,0.1554,18738452.0,09:32:00,13:13:27,27,55.000908,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22656,20191219,603998.SH,方盛制药,8.56,10.03,9.38,3.8398,0.2608,9493896.0,13:53:51,14:18:45,15,51.017030,U
22657,20191008,603998.SH,方盛制药,7.90,10.03,9.33,11.4474,0.3942,13241980.0,09:45:07,10:03:34,21,51.834454,U
22658,20190910,603998.SH,方盛制药,8.47,10.00,11.56,5.7050,0.5075,18275720.0,11:22:25,13:04:35,11,49.035870,U
22659,20190826,603998.SH,方盛制药,7.95,9.96,13.14,1.1053,0.1299,4389990.0,10:40:57,14:34:25,82,51.977028,U


#### 涨跌停强度分析
+ 接口：limit_list
+ 描述：获取每日涨跌停股票统计，包括封闭时间和打开次数等数据，帮助用户快速定位近期强（弱）势股，以及研究超短线策略。
+ 限量：单次最大1000，总量不限制

In [20]:

#获取时间段统计信息-2019年至今
dfll = pro.limit_list(start_date=sdate, end_date=edate,limit_type='U')
dfll[(dfll.trade_date=='20200327')&(dfll.ts_code=='603157.SH')]
#dfll.columns=llCname
#dfll[dfll.open_times>20].sort_values(by='open_times',ascending=False)

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
26,20200327,603157.SH,拉夏贝尔,4.49,10.05,8.33,45.6151,2.2248,14564513.0,09:41:25,09:53:34,1,56.342747,U


ValueError: Length mismatch: Expected axis has 14 elements, new values have 10 elements

In [33]:
df1227=pro.limit_list(trade_date='20191227',limit_type='U')
df1227

,trade_date,ts_code,name,close,pct_chg,amp,fc_ratio,fl_ratio,fd_amount,first_time,last_time,open_times,strth,limit
0,20191227,002972.SZ,N科安达,16.55,44.04,24.02,4574.9153,15.2450,111215744.0,09:25:03,09:25:03,0,100.000000,U
1,20191227,300807.SZ,天迈科技,45.11,10.00,0.00,5250.9064,32.7752,251343408.0,09:25:03,09:25:03,0,100.000000,U
2,20191227,300173.SZ,智慧松德,6.49,10.00,0.00,625.5034,4.1249,117704736.0,09:25:03,09:25:03,0,86.892620,U
3,20191227,603598.SH,引力传媒,14.26,10.03,0.00,600.8521,4.5545,173967728.0,09:25:04,09:25:04,0,86.029820,U
4,20191227,002163.SZ,中航三鑫,5.16,10.02,0.00,440.5163,1.5673,64977816.0,09:25:03,09:25:03,0,80.418070,U
5,20191227,000927.SZ,一汽夏利,5.83,10.00,0.00,258.5021,1.4198,132043528.0,09:25:03,09:25:03,0,74.047570,U
6,20191227,000955.SZ,欣龙控股,6.77,10.08,0.00,230.6344,4.1297,150303344.0,09:25:03,09:25:03,0,73.072205,U
7,20191227,600485.SH,*ST信威,2.77,4.92,0.00,200.4771,0.8790,46139556.0,09:25:02,09:25:02,0,72.016700,U
8,20191227,300787.SZ,海能实业,90.38,10.00,6.91,47.0713,5.5294,106047280.0,10:19:00,10:19:00,0,67.897125,U
9,20191227,600652.SH,*ST游久,2.27,5.09,1.39,40.6434,0.3816,7212986.0,09:25:04,09:25:04,0,67.693410,U


#### 查看涨跌停价格


In [69]:
#获取单个股票数据
df07limit = pro.stk_limit(ts_code='603157.SH', start_date='20190724', end_date='20200117')
df07limit

,trade_date,ts_code,up_limit,down_limit
0,20200117,603157.SH,7.28,5.96
1,20200116,603157.SH,6.62,5.42
2,20200115,603157.SH,6.55,5.36
3,20200114,603157.SH,6.51,5.33
4,20200113,603157.SH,6.49,5.31
...,...,...,...,...
116,20190730,603157.SH,6.70,5.48
117,20190729,603157.SH,6.71,5.49
118,20190726,603157.SH,6.68,5.46
119,20190725,603157.SH,6.71,5.49


In [7]:
df07daily = pro.daily(ts_code='603157.SH', start_date='20190724', end_date='20200117')
df07daily

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,603157.SH,20200117,6.44,6.44,5.96,6.08,6.62,-0.54,-8.1571,283475.93,174131.725
1,603157.SH,20200116,6.10,6.62,6.08,6.62,6.02,0.60,9.9668,376776.85,246544.025
2,603157.SH,20200115,5.96,6.19,5.87,6.02,5.95,0.07,1.1765,136810.00,83169.196
3,603157.SH,20200114,6.05,6.10,5.76,5.95,5.92,0.03,0.5068,146250.43,86685.683
4,603157.SH,20200113,6.00,6.09,5.86,5.92,5.90,0.02,0.3390,103887.92,61970.398
...,...,...,...,...,...,...,...,...,...,...,...
116,603157.SH,20190730,6.09,6.16,6.06,6.09,6.09,0.00,0.0000,20061.21,12270.988
117,603157.SH,20190729,6.10,6.13,6.05,6.09,6.10,-0.01,-0.1639,17948.67,10911.368
118,603157.SH,20190726,6.07,6.13,6.02,6.10,6.07,0.03,0.4942,21577.00,13131.268
119,603157.SH,20190725,6.11,6.13,6.04,6.07,6.10,-0.03,-0.4918,27393.46,16656.479


In [71]:
df07daily.drop(columns=['change','pct_chg','vol','amount'],inplace=True)
df07=pd.merge(df07daily,df07limit,on='trade_date')
df07.drop(columns=['ts_code_y','ts_code_x'],inplace=True)
df07                  

,trade_date,open,high,low,close,pre_close,up_limit,down_limit
0,20200117,6.44,6.44,5.96,6.08,6.62,7.28,5.96
1,20200116,6.10,6.62,6.08,6.62,6.02,6.62,5.42
2,20200115,5.96,6.19,5.87,6.02,5.95,6.55,5.36
3,20200114,6.05,6.10,5.76,5.95,5.92,6.51,5.33
4,20200113,6.00,6.09,5.86,5.92,5.90,6.49,5.31
...,...,...,...,...,...,...,...,...
116,20190730,6.09,6.16,6.06,6.09,6.09,6.70,5.48
117,20190729,6.10,6.13,6.05,6.09,6.10,6.71,5.49
118,20190726,6.07,6.13,6.02,6.10,6.07,6.68,5.46
119,20190725,6.11,6.13,6.04,6.07,6.10,6.71,5.49


In [72]:
ws=df07[(df07.high==df07.up_limit)|(df07.low==df07.down_limit)]
ws

,trade_date,open,high,low,close,pre_close,up_limit,down_limit
0,20200117,6.44,6.44,5.96,6.08,6.62,7.28,5.96
1,20200116,6.10,6.62,6.08,6.62,6.02,6.62,5.42
6,20200109,5.54,6.07,5.54,5.99,5.52,6.07,4.97
32,20191203,4.30,4.74,4.28,4.74,4.31,4.74,3.88
34,20191129,4.29,4.66,4.29,4.43,4.77,5.25,4.29
35,20191128,4.77,4.77,4.77,4.77,5.30,5.83,4.77
36,20191127,5.62,5.78,5.25,5.30,5.25,5.78,4.73
37,20191126,5.11,5.25,5.03,5.25,4.77,5.25,4.29
38,20191125,4.38,4.77,4.33,4.77,4.34,4.77,3.91


In [73]:
ws['flag'] = '最低价=跌停价'
#df.loc[df['name'] == '', 'x4'] = 0
ws.loc[ws.high == ws.up_limit, 'flag'] = '最高价=涨停价'
#ws['flag'] = ws.apply(lambda x: '最高价=涨停价'
#                     if ws.high == ws.up_limit else '最低价=跌停价',
#                      axis=1)

#ws['flag']=np.where(ws.high==ws.up_limit,'最高价=涨停价','最低价=跌停价')
ws.sort_values(by='trade_date')

/Users/bsg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/bsg/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,trade_date,open,high,low,close,pre_close,up_limit,down_limit,flag
38,20191125,4.38,4.77,4.33,4.77,4.34,4.77,3.91,最高价=涨停价
37,20191126,5.11,5.25,5.03,5.25,4.77,5.25,4.29,最高价=涨停价
36,20191127,5.62,5.78,5.25,5.30,5.25,5.78,4.73,最高价=涨停价
35,20191128,4.77,4.77,4.77,4.77,5.30,5.83,4.77,最低价=跌停价
34,20191129,4.29,4.66,4.29,4.43,4.77,5.25,4.29,最低价=跌停价
32,20191203,4.30,4.74,4.28,4.74,4.31,4.74,3.88,最高价=涨停价
6,20200109,5.54,6.07,5.54,5.99,5.52,6.07,4.97,最高价=涨停价
1,20200116,6.10,6.62,6.08,6.62,6.02,6.62,5.42,最高价=涨停价
0,20200117,6.44,6.44,5.96,6.08,6.62,7.28,5.96,最低价=跌停价


In [57]:
data = {
    'amount': [100, 200, 300, 400, 500],
    'name': ['', '商品1', '商品3', '', '商品3']
}
test= pd.DataFrame(data)
test

,amount,name
0,100,
1,200,商品1
2,300,商品3
3,400,
4,500,商品3


In [58]:
 test['x2'] = np.where(test['name'] == '', 0, test['amount'])
 test

,amount,name,x2
0,100,,0
1,200,商品1,200
2,300,商品3,300
3,400,,0
4,500,商品3,500


In [52]:
ws.drop(columns='ts_code_x')
ws

,trade_date,open,high,low,close,pre_close,up_limit,down_limit
28,20191209,9.58,10.56,9.50,10.56,9.60,10.56,8.64
38,20191125,10.50,10.51,9.32,9.96,10.36,11.40,9.32
45,20191114,11.06,11.06,10.30,11.06,10.05,11.06,9.05
46,20191113,9.15,10.05,9.15,10.05,9.14,10.05,8.23
67,20191015,10.45,10.45,9.46,9.64,9.50,10.45,8.55
69,20191011,9.00,9.80,8.86,9.60,8.91,9.80,8.02
75,20190926,8.51,8.88,7.73,8.63,8.59,9.45,7.73
76,20190925,8.27,8.90,8.25,8.59,8.09,8.90,7.28
95,20190828,10.40,10.40,9.18,9.18,10.20,11.22,9.18
96,20190827,10.20,10.20,10.20,10.20,9.27,10.20,8.34


,ts_code_x,trade_date,open,high,low,close,pre_close,up_limit,down_limit
28,000007.SZ,20191209,9.58,10.56,9.50,10.56,9.60,10.56,8.64
38,000007.SZ,20191125,10.50,10.51,9.32,9.96,10.36,11.40,9.32
45,000007.SZ,20191114,11.06,11.06,10.30,11.06,10.05,11.06,9.05
46,000007.SZ,20191113,9.15,10.05,9.15,10.05,9.14,10.05,8.23
67,000007.SZ,20191015,10.45,10.45,9.46,9.64,9.50,10.45,8.55
69,000007.SZ,20191011,9.00,9.80,8.86,9.60,8.91,9.80,8.02
75,000007.SZ,20190926,8.51,8.88,7.73,8.63,8.59,9.45,7.73
76,000007.SZ,20190925,8.27,8.90,8.25,8.59,8.09,8.90,7.28
95,000007.SZ,20190828,10.40,10.40,9.18,9.18,10.20,11.22,9.18
96,000007.SZ,20190827,10.20,10.20,10.20,10.20,9.27,10.20,8.34


In [19]:

# 获取时间段统计信息
df07limit_list = pro.limit_list(
    ts_code='603157.SH,000007.SZ', start_date='20190724', end_date='20200118')
df07limit_list.drop(columns='ts_code',inplace=True)
df07limit_list.rename(columns=llCname)

,trade_date,name,close,涨跌幅,振幅,封单额/日成交额,封单手数/流通股本,封单额,首次涨停,最后封板时,打开次数,涨跌停强度,D跌停U涨停
0,20200116,拉夏贝尔,6.62,9.97,8.97,4.5182,1.1541,11139474.0,09:44:36,14:53:37,63,51.539530,U
1,20191209,全新好,10.56,10.00,11.04,11.4832,0.5090,16605601.0,14:50:36,14:52:30,2,50.133770,U
2,20191203,拉夏贝尔,4.74,9.98,10.67,10.9873,1.7947,12403186.0,14:32:15,14:32:51,1,50.518463,U
3,20191128,拉夏贝尔,4.77,-10.00,0.00,237.3472,7.8295,54451688.0,09:25:05,09:25:05,0,73.307150,D
4,20191126,拉夏贝尔,5.25,10.06,4.61,54.8017,3.0743,23532600.0,09:32:17,09:32:17,0,71.023895,U
5,20191125,拉夏贝尔,4.77,9.91,10.14,60.3787,2.1310,14820657.0,09:43:03,09:43:03,0,61.004425,U
6,20191114,全新好,11.06,10.05,7.56,2.1534,0.1978,6760303.5,09:25:03,14:49:24,4,57.665226,U
7,20191113,全新好,10.05,9.96,9.85,11.6980,0.3910,12141405.0,10:25:45,10:33:30,8,51.339348,U
8,20190828,全新好,9.18,-10.00,11.96,1.5870,0.2383,6758775.0,13:20:42,14:22:21,21,48.210617,D
9,20190827,全新好,10.20,10.03,0.00,2400.6067,11.7163,369215008.0,09:25:03,09:25:03,0,100.000000,U
